In [1]:
import pickle
from multiprocessing import cpu_count
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import requests
from bs4 import BeautifulSoup
import numpy as np

In [35]:
def download_link_verbos(url):
    lnk_verbos = []
    r = requests.get(url)
    if r.status_code == 200:
        soup = BeautifulSoup(r.content)
        links = soup.find_all("a")
        for link in links:
            href = link.get("href").replace("/verbos-populares/", "").replace("/verbos-regulares", "").replace("/verbos-irregulares", "")
            if "verbo" in href:
                lnk_verbos += [href]
    return lnk_verbos


lnks = []
with ProcessPoolExecutor(max_workers=cpu_count()) as exc:
    links = [
        "https://www.conjugacao.com.br/verbos-regulares/",
        "https://www.conjugacao.com.br/verbos-irregulares/"
        "https://www.conjugacao.com.br/verbos-populares/"
    ] + [f"https://www.conjugacao.com.br/verbos-populares/{i}/" for i in range(2, 51)]
    link_verbos = list(exc.map(download_link_verbos, links, chunksize=10))
    for lnk in link_verbos:
        lnks += lnk

In [41]:
lnks = sorted(lnks)
print(len(lnks))

5050


In [42]:
def download_verbos(link):
    verbo = link.replace("verbo-", "").replace("/", "")
    conjugacoes = []
    r = requests.get(f"https://www.conjugacao.com.br/{link}")
    if r.status_code == 200:
        soup = BeautifulSoup(r.content)
        conjugacoes = [conjugacao.text for conjugacao in soup.find_all("span", {"class": "f"})]
    return {verbo: conjugacoes}

with ProcessPoolExecutor(max_workers=cpu_count()) as exc:
    verbos = list(exc.map(download_verbos, lnks, chunksize=100))
    with open("verbs.pkc", "wb") as outfile:
        pickle.dump(verbos, outfile)

In [46]:
with open("verbs.pkc", "rb") as infile:
    verbos = pickle.load(infile)

In [47]:
verbos_a = {}
for verb in verbos:
    for v in list(verb.values())[0][:10]:
        verbos_a[v] = list(verb.keys())[0]
# verbs = {}
# keys_verbs = []
# for verb in verbos:
#     keys_verbs += list(verb.values())[0]
#     verbs.update(verb)
# for k, v in verbs.items():
#     print(k)
# print(np.asarray(keys_verbs).ravel()[:350])
# print(verbos[0].update(verbos[1]))

In [48]:
verbos_a["ativado"]

'ativar'